In [ ]:
# In the root of the repo run "pip install --editable ."

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

# Cleaning Event Detection

Author: Clemens Jochum

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from src.data.import_data import import_df_from_zip_pkl
from src.data.make_dataset import downsample_dataframe
from src.features.build_features import find_true_cleaning_events
from src.models.detect_soiling import detect_cleaning_events_with_rolling_avg, get_accuracy_of_predicted_cleaning_events

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

#plt.rcParams["figure.figsize"] = [10, 5]
#plt.rcParams['figure.dpi'] = 250
#'font.size': 10, 'savefig.dpi': 72, 'figure.subplot.bottom': 0.125

### Import Datasets

Use one (random) dataset for each type of synthetic data.

In [ ]:
path_to_data_raw = '../data/raw/'

list_data_keys = ['basic', 'soil', 'soil_weather', 'weather']
list_index_random = [1] * len(list_data_keys)
#list_index_random = np.random.randint(50, size=len(list_data_keys))

dict_index_dataset = {}
for (data_key, dataset_index) in zip(list_data_keys, list_index_random):
    dict_index_dataset[data_key] = dataset_index
    
dict_data_paths = {}
for data_key in list_data_keys:
    dict_data_paths[data_key] = '{0:s}/synthetic_{1:s}.zip'.format(path_to_data_raw, data_key)

In [ ]:
dict_df = {}
for data_key in list_data_keys:
    dict_df[data_key] = {}
    dict_df[data_key]['T'] = import_df_from_zip_pkl(dict_data_paths[data_key], dict_index_dataset[data_key])

### Resample Datasets

Downsample minutes ('T') to hours ('H'), days ('D'), weeks ('W'), months ('MS'), and years ('AS') using mean.

In [ ]:
list_resample_rules = ['H', 'D', 'W', 'MS', 'AS']

dict_label_resample_rules = {
    'T' : 'minute',
    'H' : 'hour',
    'D' : 'day',
    'W' : 'week',
    'MS' : 'month',
    'AS' : 'year',
}

dict_timeunits_per_year = {
    'MS' : 12,
    'W' : 52,
    'D' : 365,
    'H' : 24 * 365,
    'T' : 60 * 24 * 365,
}

In [ ]:
for data_key in list_data_keys:
    for resample_rule in list_resample_rules:
        #dict_df[data_key]['T'] = remove_clipping_with_universal_window(dict_df[data_key]['T'])
        dict_df[data_key][resample_rule] = downsample_dataframe(dict_df[data_key]['T'],
                                                                clip_method = 'universal',
                                                                night_method = 'basic',
                                                                offset = resample_rule,)

### Detect cleaning events

Detect the cleaning event from a 'synthetic_soil' time-series. Does not work on 'synthetic_soil_weather'.

In [ ]:
df = dict_df['soil']['D']
#df = dict_df['soil_weather']['D']
cleaning_profile, cleaning_events_index, cleaning_events_height = detect_cleaning_events_with_rolling_avg(df['Power'])

plt.rcParams["figure.figsize"] = [10, 10]
fig, axarr = plt.subplots(4, sharex=True)

axarr[0].plot(df['Soiling'].to_numpy())
axarr[1].plot(cleaning_profile)
axarr[2].plot(np.diff(cleaning_profile))
axarr[3].plot(df['Power'].to_numpy())

for pos in cleaning_events_index:
    axarr[0].axvline(x=pos-0.0, color='k', linestyle='--', linewidth=0.5)
    axarr[1].axvline(x=pos-1.0, color='k', linestyle='--', linewidth=0.5)
    axarr[2].axvline(x=pos-1.5, color='k', linestyle='--', linewidth=0.5)
    axarr[3].axvline(x=pos-0.0, color='k', linestyle='--', linewidth=0.5)

xmin, xmax = 0, df['Power'].size
#xmin, xmax = 1000, 1100
#xmin, xmax = 1270, 1280


list_ylabel = ['soiling ratio', 'cleaning profile', 'd(cleaning profile)/dt', 'Power [kW]']
for i in range(4): 
    axarr[i].set_xlim([xmin, xmax])
    axarr[i].set_ylabel(list_ylabel[i])
     
axarr[3].set_xlabel('time [d]')
        
axarr[1].set_xlim([xmin, xmax])
axarr[2].set_xlim([xmin, xmax])
axarr[3].set_xlim([xmin, xmax])

plt.show()

In [ ]:
help(detect_cleaning_events_with_rolling_avg)

### Get accuracy from cleaning events

Detect the cleaning event from a 'synthetic_soil' time-series. Does not work on 'synthetic_soil_weather'.

In [ ]:
dict_accuracy = get_accuracy_of_predicted_cleaning_events(df)

In [ ]:
dict_accuracy

In [ ]:
help(get_accuracy_of_predicted_cleaning_events)